In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.7
)

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Give a user profile with name, age and city."
)

In [9]:
chain = prompt | llm

In [10]:
responce = chain.invoke({})  # type: ignore

In [11]:
print(responce.content)

Name: Sarah Johnson
Age: 28
City: New York City


In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(temperature=0.7)

prompt = ChatPromptTemplate.from_template("What is the capital of {country}?")

chain = prompt | llm | StrOutputParser()

responce = chain.invoke({"country": "India"})  # type: ignore
print(responce)

The capital of India is New Delhi.


In [13]:
type(responce)

str

In [14]:
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}?")

chain = prompt | llm | StrOutputParser()

responce = chain.invoke({"topic": "Coder"})  # type: ignore
print(responce)

Why do coders always mix up Christmas and Halloween?

Because Oct 31 == Dec 25!


# **JSON OUTPUT PARSER**

In [16]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(temperature=0.7)

parser = JsonOutputParser()
format_instructions = parser.get_format_instructions()

prompt_template = ChatPromptTemplate.from_template(
    """Extract the name of country and capital from the following text about a country.
    Text: {country_info}
    {format_instructions}"""
)

prompt = prompt_template.partial(format_instructions=format_instructions)
chain = prompt | llm | parser

country_info = "India is a country in South Asia. Its capital is New Delhi."
responce = chain.invoke({"country_info": country_info})  # type: ignore

print(responce)

{'country': 'India', 'capital': 'New Delhi'}


# **StructuredOutputParser**

In [18]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

model = ChatOpenAI(temperature=0.7)

response_schemas = [
    ResponseSchema(name="name", description="The name of the person"),
    ResponseSchema(name="age", description="The age of the person"),
    ResponseSchema(name="city", description="The city where the person lives"),
]

parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = parser.get_format_instructions()

prompt_template = ChatPromptTemplate.from_template(
    """Extract information about a person from the following text.
    Description: {person_description}
    {format_instructions}"""
)

prompt = prompt_template.partial(format_instructions=format_instructions)

chain = prompt | model | parser

person_description = "John is a 30-year-old software engineer living in San Francisco."

response = chain.invoke({"person_description": person_description})  # type: ignore
print(response)

{'name': 'John', 'age': '30', 'city': 'San Francisco'}


# **Pydentic Output Parser**

In [26]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Optional
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

class Product(BaseModel):
    name: str = Field(description="The name of the product")
    price: float = Field(description="The price of the product")

parser = PydanticOutputParser(pydantic_object=Product, format_instructions = "Return a JSON object with the product details.")

prompt_template = ChatPromptTemplate.from_template(
    "Provide product details in JSON. \n{format_instructions}"
)

prompt = prompt_template.partial(format_instructions=parser.get_format_instructions())
llm = ChatOpenAI(temperature=0.7)

chain = prompt | llm | parser


In [27]:
print(chain.invoke({}))

name='iPhone 12' price=999.99


In [30]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel

class WeatherInfo(BaseModel):
    city: str
    temperature: float
    condition: str 


parser = PydanticOutputParser(pydantic_object=WeatherInfo)




prompt = PromptTemplate(
    template="Provide current weather info.\n{format_instructions}\nCity: {city}",
    input_variables=["city"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)


llm = ChatOpenAI(temperature=0.3, model="gpt-3.5-turbo")


chain = LLMChain(
    llm=llm,
    prompt=prompt,
 
)

In [31]:
city_input = "Bangalore"

raw_input= chain.run({"city": city_input})
print(raw_input)
print(parser.parse(raw_input))

C:\Users\vikas\AppData\Local\Temp\ipykernel_30908\839694582.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  raw_input= chain.run({"city": city_input})


{
  "city": "Bangalore",
  "temperature": 27,
  "condition": "Partly Cloudy"
}
city='Bangalore' temperature=27.0 condition='Partly Cloudy'


# **Enum**

In [32]:
from langchain.output_parsers import EnumOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from enum import Enum

class Color(Enum):
    RED = "red"
    GREEN = "green"
    BLUE = "blue"
    
    
parser = EnumOutputParser(enum=Color)

prompt = PromptTemplate(
    template="What is your favorite color? \n{format_instructions}",
    input_variables=[],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

model = ChatOpenAI(temperature=0.7)

output = model.invoke(prompt.format())  # type: ignore


parsed = parser.parse(output.content.lower())
print(f"Raw output: {output.content}")
print(f"Parsed output: {parsed}")

Raw output: Blue
Parsed output: Color.BLUE


In [33]:
from langchain.output_parsers import DatetimeOutputParser

parser = DatetimeOutputParser()
prompt = PromptTemplate(
    template="What is the current date and time?\n{format_instructions}\n",
    input_variables=[],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

model = ChatOpenAI(temperature=0)
output = model.invoke(prompt.format())
parsed = parser.parse(output.content)
print(parsed)

2022-01-31 15:23:45.123456


In [34]:
from langchain.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()
prompt = PromptTemplate(
    template="List three fruits.\n{format_instructions}\n",
    input_variables=[],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
model = ChatOpenAI(temperature=0)
output = model.invoke(prompt.format())

parsed = parser.parse(output.content)
print(parsed)

['apple', 'banana', 'orange']


In [35]:
print("The End")

The End
